# Spooky Author Identification


Kaggle Project

https://www.kaggle.com/c/spooky-author-identification

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from keras.models import Sequential
from keras.layers.recurrent import LSTM, GRU
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.embeddings import Embedding
from keras.layers.normalization import BatchNormalization
from keras.utils import np_utils
from sklearn import preprocessing, decomposition, model_selection, metrics, pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from keras.layers import GlobalMaxPooling1D, Conv1D, MaxPooling1D, Flatten, Bidirectional, SpatialDropout1D
from keras.preprocessing import sequence, text
from keras.callbacks import EarlyStopping
from nltk import word_tokenize
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

Using TensorFlow backend.


In [2]:
ls data

sample_submission.csv  test.csv               train.csv


In [3]:
df = pd.read_csv('data/train.csv')

In [4]:
df.shape

(19579, 3)

In [5]:
df.head()

,id,text,author
0,id26305,"This process, however, afforded me no means of...",EAP
1,id17569,It never once occurred to me that the fumbling...,HPL
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP
3,id27763,How lovely is spring As we looked from Windsor...,MWS
4,id12958,"Finding nothing else, not even gold, the Super...",HPL


In [6]:
df_test = pd.read_csv('data/test.csv')

In [7]:
df_test.shape

(8392, 2)

In [8]:
df_test.head()

,id,text
0,id02310,"Still, as I urged our leaving Ireland with suc..."
1,id24541,"If a fire wanted fanning, it could readily be ..."
2,id00134,And when they had broken down the frail door t...
3,id27757,While I was thinking how I should possibly man...
4,id04081,I am not sure to what limit his knowledge may ...


In [9]:
df.isnull().sum()

id        0
text      0
author    0
dtype: int64

In [10]:
df.author.value_counts()

EAP    7900
MWS    6044
HPL    5635
Name: author, dtype: int64

## Test TF-IDF Model

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.svm import LinearSVC, SVC
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import accuracy_score

X = df.text
y = df.author

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)


# LinearSVC

text_clf = Pipeline([('tfidf',TfidfVectorizer()),('clf',LinearSVC())])
text_clf.fit(X_train,y_train)

y_pred = text_clf.predict(X_test)
print(classification_report(y_test,y_pred))
print ("Accuracy: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         EAP       0.82      0.85      0.83      1570
         HPL       0.83      0.82      0.82      1071
         MWS       0.85      0.83      0.84      1275

    accuracy                           0.83      3916
   macro avg       0.83      0.83      0.83      3916
weighted avg       0.83      0.83      0.83      3916

Accuracy:  0.832482124616956


In [12]:
# Naive Bayes

text_clf = Pipeline([('tfidf',TfidfVectorizer()),('nb',MultinomialNB())])
text_clf.fit(X_train,y_train)

y_pred = text_clf.predict(X_test)
print(classification_report(y_test,y_pred))
print ("Accuracy: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         EAP       0.74      0.92      0.82      1570
         HPL       0.93      0.67      0.78      1071
         MWS       0.87      0.80      0.83      1275

    accuracy                           0.81      3916
   macro avg       0.84      0.80      0.81      3916
weighted avg       0.83      0.81      0.81      3916

Accuracy:  0.8146067415730337


In [13]:
# Logistic Regression

text_clf = Pipeline([('tfidf',TfidfVectorizer()),('lineara',LogisticRegression(C=1.0))])
text_clf.fit(X_train,y_train)

y_pred = text_clf.predict(X_test)
print(classification_report(y_test,y_pred))
print ("Accuracy: ", accuracy_score(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


              precision    recall  f1-score   support

         EAP       0.78      0.86      0.82      1570
         HPL       0.80      0.77      0.79      1071
         MWS       0.86      0.78      0.82      1275

    accuracy                           0.81      3916
   macro avg       0.81      0.80      0.81      3916
weighted avg       0.81      0.81      0.81      3916

Accuracy:  0.8097548518896833


In [14]:
# Random Forest

from sklearn.ensemble import RandomForestClassifier

text_clf = Pipeline([('tfidf',TfidfVectorizer()),('rf',RandomForestClassifier())])
text_clf.fit(X_train,y_train)

y_pred = text_clf.predict(X_test)
print(classification_report(y_test,y_pred))
print ("Accuracy: ", accuracy_score(y_test, y_pred))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


              precision    recall  f1-score   support

         EAP       0.57      0.81      0.67      1570
         HPL       0.62      0.48      0.54      1071
         MWS       0.71      0.47      0.57      1275

    accuracy                           0.61      3916
   macro avg       0.63      0.59      0.59      3916
weighted avg       0.63      0.61      0.60      3916

Accuracy:  0.6118488253319714


In [15]:
# XGBoost Classifier

text_clf = Pipeline([('tfidf',TfidfVectorizer()),('xgb',xgb.XGBClassifier())])
text_clf.fit(X_train,y_train)

y_pred = text_clf.predict(X_test)
print(classification_report(y_test,y_pred))
print ("Accuracy: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         EAP       0.60      0.82      0.70      1570
         HPL       0.66      0.55      0.60      1071
         MWS       0.74      0.51      0.60      1275

    accuracy                           0.65      3916
   macro avg       0.67      0.63      0.63      3916
weighted avg       0.66      0.65      0.64      3916

Accuracy:  0.6465781409601634


LinearSVC gives the best result.

## Test CountVector

In [19]:
X = df.text
y = df.author

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

text_clf = Pipeline([('vectorizer',CountVectorizer()),
                    ('clf',LinearSVC())])

text_clf.fit(X_train,y_train)
y_pred = text_clf.predict(X_test)
print(classification_report(y_test,y_pred))
print ("Accuracy: ", accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

         EAP       0.79      0.84      0.81      1570
         HPL       0.82      0.76      0.79      1071
         MWS       0.82      0.80      0.81      1275

    accuracy                           0.80      3916
   macro avg       0.81      0.80      0.80      3916
weighted avg       0.80      0.80      0.80      3916

Accuracy:  0.8038815117466803


## Combine Train and Test for TF-IDF

## Submission 1

In [16]:
df_combine = df.text.append(df_test.text).reset_index(drop=True)

vector = TfidfVectorizer()
x_combine = vector.fit_transform(df_combine)
X_combine = x_combine.toarray()

In [17]:
df_combine.shape

(27971,)

In [18]:
X_train = X_combine[:19579,]
y_train = df.author
print(X_train.shape)
print(y_train.shape)

(19579, 28300)
(19579,)


In [19]:
from sklearn.calibration import CalibratedClassifierCV, calibration_curve


svm = LinearSVC()
clf = CalibratedClassifierCV(svm) 

clf.fit(X_train, y_train)

X_test = X_combine[19579:,]
y_proba = clf.predict_proba(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [20]:
y_proba

array([[0.29637443, 0.01893782, 0.68468775],
       [0.86554903, 0.12853329, 0.00591768],
       [0.13378341, 0.84854837, 0.01766822],
       ...,
       [0.81650468, 0.07109689, 0.11239843],
       [0.36871099, 0.01101317, 0.62027584],
       [0.34461232, 0.6539764 , 0.00141128]])

In [21]:
df_test['EAP'] = y_proba[:, 0]
df_test['HPL'] = y_proba[:, 1]
df_test['MWS'] = y_proba[:, 2]

df_test.head()

,id,text,EAP,HPL,MWS
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.296374,0.018938,0.684688
1,id24541,"If a fire wanted fanning, it could readily be ...",0.865549,0.128533,0.005918
2,id00134,And when they had broken down the frail door t...,0.133783,0.848548,0.017668
3,id27757,While I was thinking how I should possibly man...,0.801296,0.182296,0.016408
4,id04081,I am not sure to what limit his knowledge may ...,0.810728,0.033744,0.155528


In [22]:
# This is for submission

df_test.drop('text', axis=1).to_csv('submission_tfidf_1.csv', index=False)

## Submission 2

In [23]:
tfidf = TfidfVectorizer(min_df=3,  max_features=None, 
            strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            stop_words = 'english')


tfidf.fit(list(df.text) + list(df_test.text))
x_train_tfidf =  tfidf.transform(df.text.values) 
x_test_tfidf  =  tfidf.transform(df_test.text.values)

In [27]:
svm = LinearSVC()
clf = CalibratedClassifierCV(svm)

clf.fit(x_train_tfidf, y)
y_proba = clf.predict_proba(x_test_tfidf)
#y_pred2 = clf.predict(x_test_tfidf)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [28]:
df_test['EAP'] = y_proba[:, 0]
df_test['HPL'] = y_proba[:, 1]
df_test['MWS'] = y_proba[:, 2]

df_test.head()

,id,text,EAP,HPL,MWS
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.270124,0.047845,0.682031
1,id24541,"If a fire wanted fanning, it could readily be ...",0.808827,0.165783,0.025390
2,id00134,And when they had broken down the frail door t...,0.333336,0.642730,0.023935
3,id27757,While I was thinking how I should possibly man...,0.762429,0.218822,0.018749
4,id04081,I am not sure to what limit his knowledge may ...,0.888960,0.062379,0.048661


In [29]:
df_test.drop('text', axis=1).to_csv('submission_tfidf_2.csv', index=False)

## Use Naive Bayes to Submit Again

In [30]:
nb = MultinomialNB()
clf = CalibratedClassifierCV(nb)

X_train = X_combine[:19579,]
y_train = df.author

clf.fit(X_train, y_train)
X_test = X_combine[19579:,]
y_proba = clf.predict_proba(X_test)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)


In [31]:
df_test['EAP'] = y_proba[:, 0]
df_test['HPL'] = y_proba[:, 1]
df_test['MWS'] = y_proba[:, 2]


df_test.head()

,id,text,EAP,HPL,MWS
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.114680,0.019834,0.865485
1,id24541,"If a fire wanted fanning, it could readily be ...",0.944095,0.043759,0.012146
2,id00134,And when they had broken down the frail door t...,0.198794,0.790262,0.010944
3,id27757,While I was thinking how I should possibly man...,0.546915,0.446467,0.006618
4,id04081,I am not sure to what limit his knowledge may ...,0.935657,0.028493,0.035851


In [32]:
df_test.drop('text', axis=1).to_csv('submission_nb1.csv', index=False)

### Below follow this notebook:

https://www.kaggle.com/abhishek/approaching-almost-any-nlp-problem-on-kaggle

In [49]:
# LabelEncoder

lbl_enc = preprocessing.LabelEncoder()
y = lbl_enc.fit_transform(df.author.values)

In [50]:
df['encoder'] = y
df.head()

,id,text,author,encoder
0,id26305,"This process, however, afforded me no means of...",EAP,0
1,id17569,It never once occurred to me that the fumbling...,HPL,1
2,id11008,"In his left hand was a gold snuff box, from wh...",EAP,0
3,id27763,How lovely is spring As we looked from Windsor...,MWS,2
4,id12958,"Finding nothing else, not even gold, the Super...",HPL,1


In [51]:
x_train, x_valid, y_train, y_valid = train_test_split(df.text.values, y, 
                                                  stratify=y, 
                                                  random_state=42, 
                                                  test_size=0.1, shuffle=True)

In [52]:
print (x_train.shape)
print (x_valid.shape)

(17621,)
(1958,)


In [59]:
# Always start with these features. They work (almost) everytime!

#tfidf = TfidfVectorizer(min_df=3,  max_features=None, 
            #strip_accents='unicode', analyzer='word',token_pattern=r'\w{1,}',
            #ngram_range=(1, 3), use_idf=1,smooth_idf=1,sublinear_tf=1,
            #stop_words = 'english')

tfidf = TfidfVectorizer()


# Fitting TF-IDF to both training and test sets (semi-supervised learning)
tfidf.fit(list(x_train) + list(x_valid))
x_train_tfidf = tfidf.transform(x_train) 
x_valid_tfidf = tfidf.transform(x_valid)

In [60]:
x_train_tfidf.shape

(17621, 25068)

In [55]:
def multiclass_logloss(actual, predicted, eps=1e-15):
    """Multi class version of Logarithmic Loss metric.
    :param actual: Array containing the actual target classes
    :param predicted: Matrix with class predictions, one probability per class
    """
    # Convert 'actual' to a binary array if it's not already:
    if len(actual.shape) == 1:
        actual2 = np.zeros((actual.shape[0], predicted.shape[1]))
        for i, val in enumerate(actual):
            actual2[i, val] = 1
        actual = actual2

    clip = np.clip(predicted, eps, 1 - eps)
    rows = actual.shape[0]
    vsota = np.sum(actual * np.log(clip))
    return -1.0 / rows * vsota

In [61]:
# Logistic Regression

clf = LogisticRegression(C=1.0)
clf.fit(x_train_tfidf, y_train)
y_pred = clf.predict_proba(x_valid_tfidf)
y_pred2 = clf.predict(x_valid_tfidf)

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))

logloss: 0.589 
accuracy: 0.816 


In [57]:
y_proba

array([[0.64570316, 0.07198908, 0.28230776],
       [0.7279392 , 0.11053272, 0.16152808],
       [0.59463897, 0.16063728, 0.24472375],
       ...,
       [0.30960304, 0.23045395, 0.45994301],
       [0.27844525, 0.19130916, 0.53024558],
       [0.12123526, 0.80918181, 0.06958293]])

In [58]:
# CountVectorizer

ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(x_train) + list(x_valid))
x_train_ctv =  ctv.transform(x_train) 
x_valid_ctv = ctv.transform(x_valid)

In [62]:
# This is better than TF-IDF???

clf = LogisticRegression(C=1.0)
clf.fit(x_train_ctv, y_train)
y_pred = clf.predict_proba(x_valid_ctv)
y_pred2 = clf.predict(x_valid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))

logloss: 0.528 
accuracy: 0.781 


In [63]:
# A simple Naive Bayes on TFIDF

clf = MultinomialNB()
clf.fit(x_train_tfidf, y_train)
y_pred = clf.predict_proba(x_valid_tfidf)
y_pred2 = clf.predict(x_valid_tfidf)

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))

logloss: 0.599 
accuracy: 0.815 


In [64]:
# Naive Bayes on Counts Data

clf = MultinomialNB()
clf.fit(x_train_ctv, y_train)
y_pred = clf.predict_proba(x_valid_ctv)
y_pred2 = clf.predict(x_valid_ctv)

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))

logloss: 0.485 
accuracy: 0.832 


Looks good, so I am going to 

## have another submission. 


In [65]:
ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)

ctv.fit(list(df.text) + list(df_test.text))
x_train_ctv =  ctv.transform(df.text.values) 
x_test_ctv = ctv.transform(df_test.text.values)

In [66]:
x_train_ctv.shape

(19579, 555112)

In [67]:
clf = MultinomialNB()
clf.fit(x_train_ctv, y)
y_pred = clf.predict_proba(x_test_ctv)
y_pred2 = clf.predict(x_test_ctv)

#print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
#print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))

In [68]:
y_pred

array([[1.28144018e-02, 2.84518742e-04, 9.86901079e-01],
       [9.99992953e-01, 3.88006538e-06, 3.16652685e-06],
       [1.92579428e-01, 8.07339206e-01, 8.13662510e-05],
       ...,
       [9.99999989e-01, 2.33448772e-09, 8.24613461e-09],
       [2.64905698e-04, 1.58882368e-06, 9.99733505e-01],
       [8.50094121e-03, 9.91497161e-01, 1.89826568e-06]])

In [69]:
df_test['EAP'] = y_pred[:, 0]
df_test['HPL'] = y_pred[:, 1]
df_test['MWS'] = y_pred[:, 2]

df_test.head()

,id,text,EAP,HPL,MWS
0,id02310,"Still, as I urged our leaving Ireland with suc...",0.012814,0.000285,9.869011e-01
1,id24541,"If a fire wanted fanning, it could readily be ...",0.999993,0.000004,3.166527e-06
2,id00134,And when they had broken down the frail door t...,0.192579,0.807339,8.136625e-05
3,id27757,While I was thinking how I should possibly man...,0.857430,0.142570,3.311800e-07
4,id04081,I am not sure to what limit his knowledge may ...,0.973802,0.019574,6.623424e-03


In [58]:
# The result is even worse!!!

df_test.drop('text', axis=1).to_csv('submission2.csv', index=False)

In [36]:
# Apply SVD, I chose 120 components. 120-200 components are good enough for SVM model.

svd = decomposition.TruncatedSVD(n_components=120)
svd.fit(x_train_tfidf)
x_train_svd = svd.transform(x_train_tfidf)
x_valid_svd = svd.transform(x_valid_tfidf)

print(x_train_svd.shape)

# Scale the data obtained from SVD. Renaming variable to reuse without scaling.

scl = preprocessing.StandardScaler()
scl.fit(x_train_svd)
x_train_svd_scl = scl.transform(x_train_svd)
x_valid_svd_scl = scl.transform(x_valid_svd)

(17621, 120)


In [ ]:
#clf = SVC(C=1.0, probability=True) # since we need probabilities
#clf.fit(x_train_svd_scl, y_train)
#y_pred = clf.predict_proba(x_valid_svd_scl)
#y_pred2 = clf.predict(x_valid_svd_scl)

#print ("logloss: %0.3f " % multiclass_logloss(y_valid, y_pred))
#print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))

In [37]:
clf = LogisticRegression(C=1.0)
clf.fit(x_train_svd_scl, y_train)
y_pred = clf.predict_proba(x_valid_svd_scl)
y_pred2 = clf.predict(x_valid_svd_scl)

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


logloss: 0.776 
accuracy: 0.674 


In [60]:
# Fitting a simple xgboost on tf-idf

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)
clf.fit(x_train_tfidf.tocsc(), y_train)
y_pred = clf.predict_proba(x_valid_tfidf.tocsc())
y_pred2 = clf.predict(x_valid_tfidf.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))

logloss: 0.784 
accuracy: 0.661 


In [62]:
print(classification_report(y_valid,y_pred2))

              precision    recall  f1-score   support

           0       0.60      0.86      0.70       790
           1       0.74      0.54      0.62       564
           2       0.76      0.52      0.62       604

    accuracy                           0.66      1958
   macro avg       0.70      0.64      0.65      1958
weighted avg       0.69      0.66      0.65      1958



In [68]:
# Fitting a simple xgboost on tf-idf

ctv = CountVectorizer(analyzer='word',token_pattern=r'\w{1,}',
            ngram_range=(1, 3), stop_words = 'english')

# Fitting Count Vectorizer to both training and test sets (semi-supervised learning)
ctv.fit(list(x_train) + list(x_valid))
x_train_ctv =  ctv.transform(x_train) 
x_valid_ctv = ctv.transform(x_valid)

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(x_train_ctv.tocsc(), y_train)

y_pred = clf.predict_proba(x_valid_ctv.tocsc())
y_pred2 = clf.predict(x_valid_ctv.tocsc())

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))
print(classification_report(y_valid,y_pred2))

logloss: 0.772 
accuracy: 0.674 
              precision    recall  f1-score   support

           0       0.60      0.86      0.71       790
           1       0.77      0.55      0.64       564
           2       0.78      0.54      0.64       604

    accuracy                           0.67      1958
   macro avg       0.72      0.65      0.66      1958
weighted avg       0.70      0.67      0.67      1958



In [65]:
x_train_ctv.shape

(19579, 555112)

In [70]:
# Fitting a simple xgboost on tf-idf svd features

clf = xgb.XGBClassifier(max_depth=7, n_estimators=200, colsample_bytree=0.8, 
                        subsample=0.8, nthread=10, learning_rate=0.1)

clf.fit(x_train_svd, y_train)
y_pred = clf.predict_proba(x_valid_svd)
y_pred2 = clf.predict(x_valid_svd)

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))
print(classification_report(y_valid,y_pred2))

logloss: 0.770 
accuracy: 0.675 
              precision    recall  f1-score   support

           0       0.68      0.69      0.68       790
           1       0.65      0.66      0.65       564
           2       0.70      0.67      0.68       604

    accuracy                           0.68      1958
   macro avg       0.67      0.67      0.67      1958
weighted avg       0.68      0.68      0.68      1958



In [ ]:
clf = linearSVC()

clf.fit(x_train_svd, y_train)
y_pred = clf.predict_proba(x_valid_svd)
y_pred2 = clf.predict(x_valid_svd)

print ("logloss: %0.3f " % multiclass_logloss(y_valid,y_pred))
print ("accuracy: %0.3f " % accuracy_score(y_valid, y_pred2))
print(classification_report(y_valid,y_pred2))